In [4]:
import numpy as np
seed = 7
np.random.seed(seed)
import dask.dataframe as dd
import pandas as pd
import pandas.core.algorithms as algos


from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
#from sklearn.cross_validation import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

import os
from scipy.sparse import csr_matrix, hstack

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.layers.advanced_activations import PReLU

In [6]:
RAW_DATA_DIR='C:\\Users\\RISHABH\\Documents\\input'

## loading data

In [7]:
#importing the data
gatrain = pd.read_csv(os.path.join(RAW_DATA_DIR,'gender_age_train.csv'),
                      )
gatest = pd.read_csv(os.path.join(RAW_DATA_DIR,'gender_age_test.csv'),
                     )
phone = pd.read_csv(os.path.join(RAW_DATA_DIR,'phone_brand_device_model.csv'))
# removing duplicate values in phone dataframe so doesn't create problems while joining dataframes
phone=phone.drop_duplicates('device_id',keep='first')

events = pd.read_csv(os.path.join(RAW_DATA_DIR, 'events.csv'),
                     parse_dates=['timestamp'],
                     infer_datetime_format=True,
                     )

appevents = pd.read_csv(os.path.join(RAW_DATA_DIR, 'app_events.csv'),
                    dtype={'is_installed':bool, 'is_active':bool})

applabels = pd.read_csv(os.path.join(RAW_DATA_DIR, 'app_labels.csv')) 
folds_5=pd.read_csv(os.path.join(RAW_DATA_DIR, 'folds_5.csv')) 
folds_10=pd.read_csv(os.path.join(RAW_DATA_DIR,'folds_10.csv')) 
labelcat = pd.read_csv(os.path.join(RAW_DATA_DIR, 'label_categories.csv'))

In [10]:
import pickle
brandencoder = pickle.load(open('brandencoder.sav','rb')) 

array(['E人E本', 'E派', 'HTC', 'LG', 'LOGO', 'Lovme', 'MIL', 'OPPO', 'PPTV',
       'SUGAR', 'TCL', 'ZOYE', 'ZUK', 'vivo', '一加', '三星', '世纪天元', '世纪星',
       '中兴', '中国移动', '丰米', '乐视', '乡米', '亚马逊', '亿通', '优米', '优语', '优购',
       '先锋', '凯利通', '努比亚', '华为', '华硕', '原点', '台电', '唯比', '唯米', '嘉源', '基伍',
       '夏新', '大Q', '大可乐', '大显', '天宏时代', '天语', '奇酷', '奥克斯', '宏碁', '宝捷讯',
       '富可视', '小杨树', '小米', '尼比鲁', '帷幄', '广信', '康佳', '德卡诺', '德赛', '恒宇丰',
       '惠普', '戴尔', '摩乐', '摩托罗拉', '斐讯', '昂达', '易派', '普耐尔', '智镁', '本为',
       '朵唯', '极米', '果米', '梦米', '欧乐迪', '欧乐酷', '欧博信', '欧奇', '欧新', '欧比',
       '欧沃', '沃普丰', '波导', '海信', '海尔', '爱派尔', '瑞米', '瑞高', '白米', '百加',
       '百立丰', '碟米', '神舟', '米奇', '米歌', '糖葫芦', '糯米', '索尼', '纽曼', '维图', '美图',
       '聆韵', '联想', '至尊宝', '艾优尼', '蓝魔', '虾米', '西米', '西门子', '语信', '诺亚信',
       '诺基亚', '谷歌', '贝尔丰', '赛博宇华', '邦华', '酷比', '酷比魔方', '酷派', '酷珀', '金星数码',
       '金立', '锤子', '长虹', '青橙', '青葱', '飞利浦', '飞秒', '首云', '魅族', '鲜米', '黑米'],
      dtype=object)

In [58]:

unique_columns=np.load('unique_columns.npy')

In [31]:
def transform(brand,model): 
    #loading data
    
    #making brand feature
    brandencoder = pickle.load(open('brandencoder.sav','rb'))
    l=[] 
    l.append(brand)
    
    
    rp=brandencoder.transform(l)  
    
    n=np.zeros(len(brandencoder.classes_)) 
    
    n[rp[0]]=1 
    sPhone = sparse.csr_matrix(n)  
    print("shape of phone {}".format(sPhone.shape)) 
    
    # making model feature
    modelencoder = pickle.load(open('modelencoder.sav','rb'))
    #converting model to labels
    modl=brand+model  
    l=[] 
    l.append(modl)  
    rm=modelencoder.transform(l)  
    n=np.zeros(len(modelencoder.classes_)) 
    
    n[rm[0]]=1  
    sModel = sparse.csr_matrix(n) 
    print("shape of model {}".format(sModel.shape))
    #model frequency feature
    model_freq = pickle.load(open('model_freq.pkl','rb'))  
    mfreq=model_freq[model]
    
    mf_encoder =pickle.load(open('mf_encoder.sav','rb'))
    l=[] 
    l.append(mfreq) 
    rmf=mf_encoder.transform(l) 
    nmfreqs = len(mf_encoder.classes_) 
    n=np.zeros(nmfreqs) 
    n[rmf[0]]=1 
    sModelfreq = sparse.csr_matrix(n) 
    print("shape of mfreq {}".format(sModelfreq.shape))
    
    #making brand frequency feature 
    brand_freq = pickle.load(open('brand_freq.pkl','rb'))
    bfreq=brand_freq[brand]
    l=[] 
    l.append(bfreq)
    bf_encoder = pickle.load(open('bf_encoder.sav','rb'))

    rbf=bf_encoder.transform(l) 
    nbfreqs = len(bf_encoder.classes_) 
    n=np.zeros(nbfreqs) 
    n[rbf[0]]=1 
    sBrandfreq = sparse.csr_matrix(n)
    print("shape of brandfreq {}".format(sBrandfreq.shape)) 
    
    #making tfidf features for brand and frequency
    
    tfv = pickle.load(open('tfv_noevent.sav','rb'))

    tf=brand+" "+model
    tfidf_test=tfv.transform([tf]) 
    print("shape of tfidf {}".format(tfidf_test.shape))
    return hstack((sPhone,sModel,sModelfreq,sBrandfreq,tfidf_test))
    
    
    
    
    

In [56]:
from scipy import sparse
def transform_events(brand,model,appid,labelid,timestamps,unique_columns): 
    #loading data to make features
    
    #making brand featurea
    brandencoder = pickle.load(open('brandencoder.sav','rb'))
    l=[] 
    l.append(brand)
    rp=brandencoder.transform(l)  
    
    n=np.zeros(len(brandencoder.classes_)) 
    
    n[rp[0]]=1 
    sPhone = sparse.csr_matrix(n) 
    print("shape of phone {}".format(sPhone.shape)) 
    
    
    #making phone model features
    modelencoder = pickle.load(open('modelencoder.sav','rb'))
    #converting model to labels
    modl=brand+model  
    l=[] 
    l.append(modl)  
    rm=modelencoder.transform(l)  
    n=np.zeros(len(modelencoder.classes_)) 
    
    n[rm[0]]=1  
    sModel = sparse.csr_matrix(n)  
    print("shape of model {}".format(sModel.shape)) 
    
    #making apps features
    appencoder = pickle.load(open('appencoder.sav','rb'))
  
    #converting apps to labels
    indapps= appencoder.transform(appid)
    napps = len(appencoder.classes_) 
    n=np.zeros(napps) 
    for i in indapps: 
        n[i]=1  
    sApps =  sparse.csr_matrix(n) 
    print("shape of apps {}".format(sApps.shape))
     
    #label features
    #converting apps to labels
    labelencoder = pickle.load(open('labelcoder.sav','rb')) 
    indlabels= labelencoder.transform(labelid)
    nlabels = len(labelencoder.classes_) 
    n=np.zeros(nlabels) 
    for i in indlabels: 
        n[i]=1  
    sLabels = sparse.csr_matrix(n)
    
    print("shape of labels {}".format(sLabels.shape))
    #model frequency feature
#model frequency feature
    model_freq = pickle.load(open('model_freq.pkl','rb'))  
    mfreq=model_freq[model]
    
    mf_encoder =pickle.load(open('mf_encoder.sav','rb'))
    l=[] 
    l.append(mfreq) 
    rmf=mf_encoder.transform(l) 
    nmfreqs = len(mf_encoder.classes_) 
    n=np.zeros(nmfreqs) 
    n[rmf[0]]=1 
    sModelfreq = sparse.csr_matrix(n) 
    print("shape of mfreq {}".format(sModelfreq.shape))
    
    #making brand frequency feature 
    brand_freq = pickle.load(open('brand_freq.pkl','rb'))
    bfreq=brand_freq[brand]
    l=[] 
    l.append(bfreq)
    bf_encoder = pickle.load(open('bf_encoder.sav','rb'))

    rbf=bf_encoder.transform(l) 
    nbfreqs = len(bf_encoder.classes_) 
    n=np.zeros(nbfreqs) 
    n[rbf[0]]=1 
    sBrandfreq = sparse.csr_matrix(n)
    
    print("shape of bfreq {}".format(sBrandfreq.shape)) 
    #number of events feature
    nevents=len(timestamps) 

    nevents = (np.log(nevents)) 
    hours=[]
    nevents=nevents/np.log(243460) 
    
    # hour feature
    for t in timestamps: 
       hours.append(t.hour) 
    n=np.zeros(24) 
    for hour in hours:
        n[hour]=1  
    sHours = sparse.csr_matrix(n) 
    print("shape of hours {}".format(sHours.shape))
    
    
     #preparing tfidf features  
    
    tfv = pickle.load(open('tfv_event.sav','rb'))
    terms=brand+" "+model 
    
    labelid=list(labelid)
    labelid.insert(0,tf) 
    terms=labelid  
    terms=list(map(str, tf))
    terms=" ".join(tf)
    
    tfidf_test=tfv.transform([terms]) 
    
    print("shape of tfidf {}".format(tfidf_test.shape))
    Xtest=hstack((sPhone,sModel,sModelfreq,sBrandfreq,sApps,sLabels,nevents,sHours,tfidf_test))  
    print(Xtest.shape)
    Xtest=Xtest.tocsc()[:,unique_columns]
    return Xtest
     
   
    
    
    

    

## First Function for model

In [87]:
from keras.models import load_model
def f1(X):  
    it = [x for x in range(1,12)] 
    val=iter(X) 
    global unique_columns
    res_dct = dict(zip(it, val))  
    eventid=res_dct[2] 
    timestamps=res_dct[3] 
    timestamps=pd.to_datetime(timestamps)
    appid=[res_dct[6]] 
    eventid=res_dct[2]
    applabel=res_dct[9]
    phone=res_dct[10] 
    model=res_dct[11]
    ypred=np.zeros((1,12))
    if eventid!=0: 
        Xte=transform(phone,model) 
        for i in range(1,11): 
            loaded_model = pickle.load(open('logistic_'+str(i)+'.sav', 'rb'))
            ypred+=loaded_model.predict_proba(Xte)
        ypred=ypred/10 
    else: 
        Xte=transform_events(phone,model,appid,applabel,timestamps,unique_columns)
        for i in range(1,11): 
            loaded_model = load_model('keras_'+str(i)+'.hdf5')
            ypred+=loaded_model.predict_proba(Xte)
        ypred=ypred/10 
    return ypred   



In [75]:
t=gatest.iloc[:2,:] 
t=test.merge(events) 
t=test.merge(appevents) 
t=test.merge(applabels) 
t=test[test.device_id==test.device_id[0]]

In [76]:
test=test.merge(phone) 


In [77]:
test

,device_id,event_id,timestamp,longitude,latitude,app_id,is_installed,is_active,label_id,phone_brand,device_model
0,1002079943728939269,460577,2016-05-03 21:06:29,0.0,0.0,5927333115845830913,True,False,549,小米,小米note
1,1002079943728939269,460577,2016-05-03 21:06:29,0.0,0.0,5927333115845830913,True,False,710,小米,小米note
2,1002079943728939269,460577,2016-05-03 21:06:29,0.0,0.0,5927333115845830913,True,False,704,小米,小米note
3,1002079943728939269,460577,2016-05-03 21:06:29,0.0,0.0,5927333115845830913,True,False,548,小米,小米note
4,1002079943728939269,460577,2016-05-03 21:06:29,0.0,0.0,5927333115845830913,True,False,172,小米,小米note
...,...,...,...,...,...,...,...,...,...,...,...
1781,1002079943728939269,3127685,2016-05-06 22:05:18,0.0,0.0,3717049149426646122,True,False,548,小米,小米note
1782,1002079943728939269,3127685,2016-05-06 22:05:18,0.0,0.0,3717049149426646122,True,False,183,小米,小米note
1783,1002079943728939269,2145937,2016-05-05 08:28:20,0.0,0.0,-3473060463208403965,True,False,1014,小米,小米note
1784,1002079943728939269,2774265,2016-05-07 09:08:31,0.0,0.0,-3473060463208403965,True,False,1014,小米,小米note


In [80]:
test=[1002079943728939269, 460577, ['2016-05-03 21:06:29','2016-05-05 08:28:20'], 0.0, 0.0, 5927333115845830913, True, False, [549,710,172], '小米', '小米note']

In [88]:
import pickle
f1(test)

shape of phone (1, 131)
shape of model (1, 1667)
shape of mfreq (1, 310)
shape of brandfreq (1, 73)
shape of tfidf (1, 1570)


array([[0.05807013, 0.05720962, 0.04567753, 0.05930158, 0.06988216,
        0.05366598, 0.117675  , 0.12713613, 0.08327792, 0.09351835,
        0.12175918, 0.11282642]])

In [ ]:
transform()

## Second function for model

In [51]:
 
def f1(device_id,phone,model,has_event=0,appid=0,applabel=0,timestamps=0,unique_columns=0,y): 
    ypred=np.zeros((1,12))
    if has_event==0: 
        Xte=transform(device_id,phone,model) 
        for i in range(1,11): 
            loaded_model = pickle.load(open('logistic_'+str(i)+'.sav', 'rb'))
            ypred+=loaded_model.predict_proba(Xte)
        ypred=ypred/10 
    else: 
        Xte=transform(device_id,phone,model,appid,applabel,timestamps,unique_columns)
        for i in range(1,11): 
            loaded_model = load_model('keras_'+str(i)+'.hdf5')
            ypred+=loaded_model.predict_proba(Xte)
        ypred=ypred/10  
        loss=log_loss(ypred,y)
    return loss   




In [92]:
from prettytable import PrettyTable
x = PrettyTable()

x.field_names = ["Model", "Parameters", "Score"]

x.add_row(["Logistic Regression", "C=0.02", 2.27])
x.add_row(["Decision Tree","min_sample_split=6 ,max_depth=6", 2.41])
x.add_row(["Xgboost", "'max_depth':6 ,'min_child_weight':5", 2.257])
x.add_row(["Keras 1", "epochs=18 batches=175", 2.24])
x.add_row(["Keras 2 different models for events and no events", "events:{epochs=80 batches=5} noevents:{epochs=30 batches=100}", 2.24])
x.add_row(["Keras 3","epochs=200 batches=5", 2.2378])
x.add_row(["Keras for events and logistic regression for noevents","keras:{epochs=20,batches=200}, lr:{C=0.016} ", 2.23704]) 
x.add_row(["ensemble model","5 model predictions",2.23714])
x.align = "l"
print(x)

+-------------------------------------------------------+---------------------------------------------------------------+---------+
| Model                                                 | Parameters                                                    | Score   |
+-------------------------------------------------------+---------------------------------------------------------------+---------+
| Logistic Regression                                   | C=0.02                                                        | 2.27    |
| Decision Tree                                         | min_sample_split=6 ,max_depth=6                               | 2.41    |
| Xgboost                                               | 'max_depth':6 ,'min_child_weight':5                           | 2.257   |
| Keras 1                                               | epochs=18 batches=175                                         | 2.24    |
| Keras 2 different models for events and no events     | events:{epochs=80 